In [1]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama



In [9]:
llm = Ollama(model="llama3.2:3b", request_timeout=60.0)


In [12]:

response = llm.complete("What is the capital of France?")
print(response)

The capital of France is Paris.


In [39]:
from llama_index.core import SimpleDirectoryReader, GPTVectorStoreIndex, StorageContext, load_index_from_storage
documents = SimpleDirectoryReader('documentacion').load_data()


In [ ]:
from llama_index.llms.